# Chapter 7 데이터 정제 및 준비 

- 해당 장에서는 결측치, 중복 데이터, 문자열 처리 그리고 다른 분석적 데이터 변환에 대한 도구들을 다룬다.

## 7.1 누락된 데이터 처리하기

- pandas의 설계 목표 중 하나는 누락 데이터를 가능한 한 쉽게 처리할 수 있도록 하는 것.
- pandas 객체의 모든 기술 통계는 누락된 데이터를 배제하고 처리함.
- pandas는 누락된 데이터를 실숫값인 NaN으로 취급함.

In [1]:
import pandas as pd
import numpy as np

In [3]:
string_data = pd.Series(['aardvark','artichoke',np.nan,'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [4]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

> 파이썬의 내장 None 값 또한 NA 값으로 취급됨.

In [5]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

#### NA 처리 메서드

<details>
<summary>NA 처리 메서드</summary>
<div markdown="1">

|인자|설명|
|:--|:--|
|dropna|누락된 데이터가 있는 축(로우,컬럼)을 제외시킨다. 어느 정도의 누락 데이터까지 용인할 것인지 지정할 수 있다.|
|fillna|누락된 데이터를 대신할 값을 채우거나 'ffill'이나 'bfill' 같은 보간 메서드를 적용한다.|
|isnull|누락되거나 NA인 값을 알려주는 불리언값이 저장된 같은 형의 객체를 반환한다.|
|notnull|isnull과 반대되는 메서드|
    
</div>
</details>

### 7.1.1 누락된 데이터 골라내기 (dropna)

> dropna를 사용하면 매우 간편하게 누락된 데이터를 골라낼 수 있다.  
> Series에 dropna 메서드를 적용하면 $\text{널이 아닌}^{\text{non-null}}$ 데이터와 색인값만 들어 있는 Series를 반환한다.

In [9]:
from numpy import nan as NA

data = pd.Series([1,NA,3.5,NA,7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [10]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

> pandas.isnull, pandas.notnull이나 불리언 색인을 사용해 직접 손으로 제거할 수도 있다.

In [11]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

> DataFrame 객체의 경우 NA를 제거하는 두 가지 방법이 있다.  
> - 별도의 지정이 없는 경우 : NA 값을 하나라도 포함하고 있는 로우나 컬럼을 제외한다.  
> - how='all' 옵션을 넘기면 모두 NA 값인 로우만 제외시킨다.

In [12]:
data = pd.DataFrame([[1.,6.5,3],[1.,NA,NA],
                     [NA,NA,NA],[NA,6.5,3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [13]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


In [14]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


> 컬럼을 제외시키는 방법도 동일하게 동작한다.  
> 옵션으로 axis = 1을 넘겨주면 된다.

In [16]:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [20]:
data.dropna(how='all', axis = 'columns')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


> DataFrame의 로우를 제외시키는 방법은 시계열 데이터에 주로 사용되는 경향이 있음.  
> 몇 개 이상의 값이 들어 있는 로우만 살펴보고 싶다면 thresh 인자에 원하는 값을 넘기면 된다.

In [22]:
df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4,1] = NA
df.iloc[:2,2] = NA

df

,0,1,2
0,-0.704895,NaN,NaN
1,0.429249,NaN,NaN
2,-1.356953,NaN,1.603586
3,0.286830,NaN,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


In [23]:
df.dropna()

,0,1,2
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


In [24]:
df.dropna(thresh=2)

,0,1,2
2,-1.356953,NaN,1.603586
3,0.286830,NaN,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


### 7.1.2 결측치 채우기 (fillna)

> 누락된 값을 제외시키지 않고 데이터 상의 '구멍'을 어떻게든 메우고 싶은 경우 fillna 메서드에 채워 넣고 싶은 값을 넘겨주면 된다.

In [26]:
df.fillna(0)

,0,1,2
0,-0.704895,0.000000,0.000000
1,0.429249,0.000000,0.000000
2,-1.356953,0.000000,1.603586
3,0.286830,0.000000,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


> fillna에 사전값을 넘겨서 각 컬럼마다 다른 값을 채울 수도 있다.

In [27]:
df.fillna({1:0.5,2:0})

,0,1,2
0,-0.704895,0.500000,0.000000
1,0.429249,0.500000,0.000000
2,-1.356953,0.500000,1.603586
3,0.286830,0.500000,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


> fillna는 `새로운 객체를 반환`하지만 inplace=True를 사용하면 `기존 객체를 변경`할 수도 있다.

In [29]:
_ = df.fillna(0, inplace = True)
df

,0,1,2
0,-0.704895,0.000000,0.000000
1,0.429249,0.000000,0.000000
2,-1.356953,0.000000,1.603586
3,0.286830,0.000000,-1.622452
4,-1.685947,1.280417,1.186311
5,0.348546,0.236392,-0.447461
6,0.852073,0.465847,-0.484564


> 재색인에서 사용 가능한 보간 메서드는 fillna 메서드에서도 사용 가능하다.  
> 예를 들어 method = "ffill", "bfill"

In [31]:
df = pd.DataFrame(np.random.randn(6,3))
df.iloc[2:,1] = NA
df.iloc[4:,2] = NA
df

,0,1,2
0,-0.982547,1.002809,1.119510
1,-0.434269,1.717283,0.730459
2,-0.615606,NaN,-1.687928
3,-0.407083,NaN,0.399892
4,0.285687,NaN,NaN
5,-1.777159,NaN,NaN


In [32]:
df.fillna(method = 'ffill')

,0,1,2
0,-0.982547,1.002809,1.119510
1,-0.434269,1.717283,0.730459
2,-0.615606,1.717283,-1.687928
3,-0.407083,1.717283,0.399892
4,0.285687,1.717283,0.399892
5,-1.777159,1.717283,0.399892


In [33]:
df.fillna(method='ffill', limit = 2)

,0,1,2
0,-0.982547,1.002809,1.119510
1,-0.434269,1.717283,0.730459
2,-0.615606,1.717283,-1.687928
3,-0.407083,1.717283,0.399892
4,0.285687,NaN,0.399892
5,-1.777159,NaN,0.399892


> 조금만 창의적으로 생각하면 fillna를 이용해서 매우 다양한 일을 할 수 있다.  
> 예를 들면 Series의 평균값이나 중간값을 전달할 수도 있다.

In [35]:
data = pd.Series([1.,NA,3.5,NA,7])
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

#### fillna 함수 인자

<details>
<summary>fillna 함수 인자</summary>
<div markdown="1">

|인자|설명|
|:--|:--|
|value|비어 있는 값을 채울 스칼라값이나 사전 형식의 객체|
|method|보간 방식. 기본적으로 'ffill'을 사용한다.|
|axis|값을 채워 넣을 축. 기본값은 axis = 0이다.|
|inplace|복사본을 생성하지 않고 호출한 객체를 변경한다. 기본값은 False다.|
|limit|값을 앞 혹은 뒤에서부터 몇 개까지 채울지 지정한다.|
    
</div>
</details>

## 7.2 데이터 변형

### 7.2.1 중복 제거하기

> DataFrame의 duplicated 메서드는 각 로우가 중복인지 아닌지 알려주는 불리언 Series를 반환한다.  
> drop_duplicates는 duplicated 배열이 False인 DataFrame을 반환한다.

In [39]:
data = pd.DataFrame({'k1': ['one','two'] * 3 + ['two'],
                     'k2': [1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [40]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [42]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


> 두 메서드는 기본적으로 모든 컬럼에 적용되며 중복을 찾아내기 위한 부분합을 따로 지정해 줄 수도 있다.  
> 새로운 컬럼을 하나 추가하고 'k1' 컬럼에 기반해서 중복을 걸러내려면 다음과 같이 한다.

In [48]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


> duplicated와 drop_duplicates는 기본적으로 처음 발견된 값을 유지함.  
> keep='last' 옵션을 넘기면 마지막으로 발견된 값을 반환한다.

In [49]:
data.drop_duplicates(['k1','k2'], keep = 'last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


### 7.2.2 함수나 매핑을 이용해서 데이터 변형하기

In [2]:
data = pd.DataFrame({'food' : ['bacon','pulled pork','bacon','Pastrami','corned beef','Bacon','pastrami','honey ham','nova lox'],
                     'ounces' : [4,3,12,6,7.5,8,3,5,6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


> 해당 육류가 어떤 동물의 고기인지 알려줄 수 있는 컬럼을 추가한다고 가정, 육류별 동물을 담고 있는 사전 데이터를 아래와 같이 작성

In [3]:
meat_to_animal = {
    'bacon' : 'pig',
    'pulled pork' : 'pig',
    'pastrami' : 'cow',
    'corned beef' : 'cow',
    'honey ham' : 'pig',
    'nova lox' : 'salmon'
}

> Series의 map 메서드는 사전류의 객체나 어떤 함수를 받을 수 있다.  
> 우선, 육류 일므에 대소문자가 섞여 있는 사소한 문제가 있으므로 str.lower 메서드를 사용해서 모두 소문자로 변경한다.

In [4]:
lowercased = data['food'].str.lower()

In [5]:
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [60]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


> 함수를 넘겨서 같은 일을 수행할 수도 있다.

In [6]:
data['food'].map(lambda x : meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### 7.2.3 값 치환하기

> fillna 메서드는 누락된 값을 채우는 작업은 일반적인 값 치환 작업.  
> map 메서드를 한 객체 안에서 값의 부분집합을 변경하는 데 사용했다면 replace 메서드는 같은 작업에 대해 좀 더 간단하고 유연한 방법을 제공함.

In [9]:
data = pd.Series([1.,-999.,2.,-999.,-1000.,3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

> replace 메서드를 이용해 pandas에서 인식할 수 있는 NA 값으로 치환해 `새로운 Series`를 생성할 수 있다. (인자로 inplace = True를 넘기지 않았다면).

In [10]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

> 여러 개의 값을 한 번에 치환하려면 하나의 값 대신 치환하려는 값의 리스트를 넘기면 된다.

In [11]:
data.replace([-999,-1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

> 치환하려는 값마다 다른 값으로 치환하려면 누락된 값 대신 새로 지정할 값의 리스트를 사용하면 된다.

In [12]:
data.replace([-999,-1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

> 두 개의 리스트 대신 사전을 이용하는 것도 가능하다.

In [13]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 7.2.4 축 색인 이름 바꾸기

> Series의 값들처럼 축 이름 역시 유사한 방식으로 함수나 새롭게 바꿀 값을 이용해서 변환할 수 있다.  
> 즉, 새로운 자료구조를 만들지 않고 그 자리에서 바로 축 이름을 변경하는 것이 가능하다.

In [14]:
data = pd.DataFrame(np.arange(12).reshape((3,4)),
                    index = ['Ohio','Colorado','New York'],
                    columns = ['one','two','three','four'])

> Series와 마찬가지로 축 색인에도 map 메서드가 있다.

In [16]:
transform = lambda x : x[:4].upper()

data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

> 대문자로 변경된 축 이름을 DataFrame의 index에 바로 대입할 수 있다.

In [17]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


> 원래 객체를 변경하지 않고 `새로운 객체를 생성`하려면 rename 메서드를 사용한다.

In [18]:
data.rename(index = str.title, columns = str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


> 특히 rename 메서드는 사전 형식의 객체를 이용해 축 이름 중 일부만 변경하는 것도 가능하다.

In [19]:
data.rename(index = {"OHIO" : "INDIANA"},
            columns = {'three' : 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [20]:
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


> rename 메서드 인자에 inplace = True 옵션을 넘겨주면 DataFrame을 직접 복사해서 index와 columns 속성을 갱신할 필요 없이 바로 변경할 수 있다.

In [21]:
data.rename(index = {'OHIO' : 'INDIANA'}, inplace = True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### 7.2.5 개별화와 양자화

- 연속형 데이터는 종종 개별로 분할하거나 그룹별로 나누기도 함

> 수업에 참여하는 학생 그룹 데이터가 있고, 나이대에 따라 분류한다고 가정하자.

In [22]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61 ,45 ,41 ,32]

> pandas의 cut 함수를 이용해 18-25, 26-35, 36-60, 61이상 그룹으로 나눌 수 있다.

In [23]:
bins = [18,25,35,60,100]
cats = pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

> pandas.cut을 통해 반환되는 객체는 Categorical이라는 특수한 객체다.  
> 이 객체는 그룹 이릅이 담긴 배열이다.  
> Categoricla 객체는 codes 속성에 있는 ages 데이터에 대한 카테고리 이름을 categories라는 배열에 내부적으로 담고 있다.

In [24]:
type(cats)

pandas.core.arrays.categorical.Categorical

In [25]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [26]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

> pd.value_counts(cats)는 pandas.cut 결과에 대한 그룹 수다.

In [27]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

> 간격을 나타내는 표기법은 중괄호(값을 포함하지 않음)로 시작해서 대괄호(값을 포함함)로 끝남.  
> right = False를 넘겨주면 중괄호 대신 대괄호 쪽이 포함되지 않도록 바꿀 수 있다.

In [28]:
pd.cut(ages, [18,26,36,61,100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

> labels 옵션으로 그룹의 이름을 직접 념겨줄 수도 있다.

In [29]:
group_names = ['Youth','YoungAdult','MiddleAged','Senior']

pd.cut(ages, bins, labels = group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

> 만약 cut함수에 명시적으로 그룹의 경곗값을 넘기지 않고 그룹의 개수를 넘겨주면 데이터에서 최솟값과 최댓값을 기준으로 `균등한 길이`의 그룹을 자동으로 계산한다.  
> 어떤 균등분포 내에서 4개의 그룹으로 나누는 경우를 생각해보자

In [30]:
data = np.random.rand(20)

pd.cut(data,4,precision = 2)    # precision = 2 옵션은 소수점 아래 2자리까지로 제한한다.

[(0.54, 0.77], (0.087, 0.31], (0.31, 0.54], (0.087, 0.31], (0.31, 0.54], ..., (0.087, 0.31], (0.087, 0.31], (0.31, 0.54], (0.77, 0.99], (0.77, 0.99]]
Length: 20
Categories (4, interval[float64]): [(0.087, 0.31] < (0.31, 0.54] < (0.54, 0.77] < (0.77, 0.99]]

> qcut 함수는 표본 변위치를 기반으로 데이터를 나눠준다.  
> cut 함수는 데이터의 분산에 따라 각각의 그룹마다 데이터 수를 다르게 나눌 수도 있다.  
> qcut 함수는 표준 변위치를 사용하기 때문에 적당히 같은 크기의 그룹으로 나눌 수 있다.

In [31]:
data = np.random.randn(1000)    # 정규분포

cats = pd.qcut(data,4)          # 4분위로 분류
cats

[(0.021, 0.673], (0.673, 3.041], (-2.972, -0.654], (0.673, 3.041], (0.021, 0.673], ..., (0.673, 3.041], (0.673, 3.041], (0.021, 0.673], (-0.654, 0.021], (0.021, 0.673]]
Length: 1000
Categories (4, interval[float64]): [(-2.972, -0.654] < (-0.654, 0.021] < (0.021, 0.673] < (0.673, 3.041]]

In [32]:
pd.value_counts(cats)

(-2.972, -0.654]    250
(-0.654, 0.021]     250
(0.021, 0.673]      250
(0.673, 3.041]      250
dtype: int64

> cut 함수와 유사하게 변위치를 직접 지정해줄 수 있다 (변위치는 0부터 1까지다).

In [34]:
cats = pd.qcut(data, [0,0.1,0.5,0.9,1.])
cats

[(0.021, 1.305], (0.021, 1.305], (-2.972, -1.256], (0.021, 1.305], (0.021, 1.305], ..., (0.021, 1.305], (0.021, 1.305], (0.021, 1.305], (-1.256, 0.021], (0.021, 1.305]]
Length: 1000
Categories (4, interval[float64]): [(-2.972, -1.256] < (-1.256, 0.021] < (0.021, 1.305] < (1.305, 3.041]]

In [35]:
pd.value_counts(cats)

(-1.256, 0.021]     400
(0.021, 1.305]      400
(-2.972, -1.256]    100
(1.305, 3.041]      100
dtype: int64

### 7.2.6 특잇값을 찾고 제외하기

In [36]:
data = pd.DataFrame(np.random.randn(1000,4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.016002,0.048567,0.004317,-0.008076
std,1.016155,1.012362,0.984672,1.043027
min,-2.910541,-3.700895,-3.129792,-3.598236
25%,-0.698165,-0.624154,-0.622816,-0.721540
50%,0.032806,0.070648,0.017522,-0.011897
75%,0.676651,0.726912,0.647156,0.676079
max,3.677838,3.264187,3.209790,2.801145


> 이 DataFrame의 한 컬럼에서 절댓값이 3을 초과하는 값을 찾아내자.

In [37]:
col = data[2]
col[np.abs(col) > 3]

470    3.209790
596   -3.129792
852    3.034472
Name: 2, dtype: float64

> 절댓값이 3을 초과하는 값이 들어 있는 모든 로우를 선택하려면 불리언 DataFrame에서 any 메서드를 사용하면 된다.

In [38]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
91,0.976900,-0.191421,-0.385661,-3.598236
307,-1.357123,-3.422233,-0.097197,0.095337
404,0.182828,-3.458526,1.046670,-1.174925
427,0.160581,3.264187,0.677020,-0.667329
470,0.705516,-0.613814,3.209790,1.748832
488,3.219851,1.852048,-1.215176,0.667982
558,-0.314690,-3.493185,-0.846214,0.556467
596,0.376707,-0.672209,-3.129792,0.082828
649,1.009898,-3.700895,-0.704360,0.184157
662,-2.082124,3.033138,1.030809,0.232117


> -3이나 3을 초과하는 값을 -3 또는 3으로 지정할 수 있다.

In [39]:
data[np.abs(data)>3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.016899,0.050265,0.004202,-0.007478
std,1.013233,1.004382,0.983500,1.041136
min,-2.910541,-3.000000,-3.000000,-3.000000
25%,-0.698165,-0.624154,-0.622816,-0.721540
50%,0.032806,0.070648,0.017522,-0.011897
75%,0.676651,0.726912,0.647156,0.676079
max,3.000000,3.000000,3.000000,2.801145


> np.sign(data)는 data 값이 양수인지 음수인지에 따라 1이나 -1이 담긴 배열을 반환한다.

In [40]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,-1.0,1.0,-1.0,1.0
2,1.0,-1.0,1.0,-1.0
3,-1.0,-1.0,1.0,1.0
4,1.0,1.0,1.0,1.0


> ### 7.2.7 치환과 임의 샘플링

> numpy.random.permutation 함수를 이용하면 Series나 DataFrame의 로우를 쉽게 임의 순서로 재배치할 수 있다.  
> 순서를 바꾸고 싶은 만큼의 길이를 permutation 함수로 넘기면 바뀐 순서가 담긴 정수 배열이 생성된다. 

In [41]:
df = pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler = np.random.permutation(5)

sampler

array([4, 3, 1, 2, 0])

> 이 배열은 iloc 기반의 색인이나 take 함수에서 사용 가능하다.

In [42]:
df.iloc[sampler]

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3


In [43]:
df.take(sampler)

,0,1,2,3
4,16,17,18,19
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11
0,0,1,2,3


> 치환 없이 일부만 임의로 선택하려면 Series나 DataFrame의 sample 메서드를 사용하면 된다.

In [44]:
df.sample(n=3)

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
0,0,1,2,3


> `반복 선택을 허용하며` 표본을 치환을 통해 생성해내려면 sample에 replace = True 옵션을 넘긴다.

In [83]:
choices = pd.Series([5,7,-1,6,4])
draws = choices.sample(n=10, replace = True)
draws

2   -1
1    7
4    4
3    6
3    6
4    4
3    6
1    7
4    4
0    5
dtype: int64

### 7.2.8 표시자/더미 변수 계산하기

- 어떤 DataFrame의 한 컬럼에 k가지의 값이 있다면 k개의 컬럼이 있는 DataFrame이나 행렬을 만들고 값으로는 1과 0을 채워 넣는 것이 더미 변수임.  
- pandas의 get_dummies는 이를 위한 함수

In [45]:
df = pd.DataFrame({'key' : ['b','b','a','c','a','b'],
                   'data1' : range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


> 표시자 DataFrame 안에 있는 컬럼에 $\text{접두어}^{\text{prefix}}$를 추가한 후 다른 데이터와 병합하고 싶을 경우 get_dummies 함수의 prefix 인자를 사용하면 이를 수행할 수 있다.

In [46]:
dummies = pd.get_dummies(df['key'], prefix = 'key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


> DataFrame의 한 로우가 여러 카테고리에 속하는 경우.  
> 자세한 내용은 14.2절 'MovieLens'의 영화 평점 데이터'에서 살펴봄.

In [47]:
mnames = ['movie_id', 'title', 'genres']

movies = pd.read_table('datasets/movielens/movies.dat',sep = '::',header = None, names =mnames, engine = 'python')
movies.head()

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


> 각 장르마다 표시자 값을 추가하기 위해 데이터 묶음에서 유일한 장르 목록을 추출함

In [48]:
all_genres = []

for x in movies.genres : 
    all_genres.extend(x.split('|'))
    
genres = pd.unique(all_genres)
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

> 표시자 DataFrame을 생성하기 위해 0으로 초기화된 DataFrame을 생성

In [49]:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns = genres)
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


> 각 영화를 순회하면서 해당하는 장르에 1을 표시해주기 위해 dummies.columns.get_indexer를 사용 (input과 일치하는 column의 위치를 반환해줌)

In [50]:
gen = movies.genres[0]
gen.split('|')

['Animation', "Children's", 'Comedy']

In [51]:
dummies.columns.get_indexer(gen.split('|'))

array([0, 1, 2], dtype=int64)

> .iloc를 이용해서 색인에 맞게 값을 대입하기

In [52]:
for i, gen in enumerate(movies.genres) : 
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i,indices] = 1

In [53]:
movies_windic = movies.join(dummies.add_prefix("Genre_"))
movies_windic.head()

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,...,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


> get_dummies와 cut 같은 이산함수를 잘 조합하면 통계 애플리케이션에서 유용하게 사용할 수 있다.

In [54]:
np.random.seed(12345)

values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [55]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 7.3 문자열 다루기

- 파이썬은 문자열이나 텍스트 처리의 용이함 덕분에 원시 데이터를 처리하는 인기 있는 언어가 됐음
- pandas는 배열 데이터 전체에 쉽게 정규 표현식을 적용하고, 누락된 데이터를 편리하게 처리할 수 있는 기능을 포함하고 있다.

### 7.3.1 문자열 객체 메서드

> 문자열 분리 (split)

In [57]:
val = "a,b,  guido"
val.split(",")

['a', 'b', '  guido']

> 공백문자 제거 (strip)

In [59]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

> 문자열 합치기 (join)

In [60]:
"::".join(pieces)

'a::b::guido'

> 부분문자열 탐색 (index, find)

In [62]:
'guido' in val

True

In [63]:
val.index(",")

1

In [65]:
val.find(":")   # index()는 문자열을 찾지 못하면 예외를 발생시키지만 find()는 -1을 반환함

-1

> 문자갯수 (count)

In [66]:
val.count(",")

2

> 문자 치환 (replace)

In [67]:
val.replace(",", "::")

'a::b::  guido'

#### 파이썬 내장 문자열 함수


<details>
<summary>파이썬 내장 문자열 함수</summary>
<div markdown="1">

|함수|설명|
|:--|:--|
|count|문자열에서 겹치지 않는 부분문자열의 개수를 반환한다.|
|endswith|문자열이 주어진 접미사로 끝날 경우 True를 반환한다.|
|startswith|문자열이 주어진 접두사로 시작할 경우 True를 반환한다.|
|join|문자열을 구분자로 하여 다른 문자열을 순서대로 이어붙인다.|
|index|부분문자열의 첫 번째 글자의 위치를 반환한다. 부분문자열이 없을 경우 ValueError 예외가 발생한다.|
|find|`첫 번째` 부분문자열의 첫 번째 글자의 위치를 반환한다. index와 유사하지만 부분문자열이 없을 경우 -1을 반환한다.|
|rfind|`마지막` 부분문자열의 첫 번째 글자의 위치를 반환한다. 부분문자열이 없을 경우 -1을 반환한다.|
|replace|문자열을 다른 문자열로 치환한다.|
|strip, rstrip, lstrip| 개행 문자를 포함한 공백 문자를 제거한다. lstrip은 문자열의 시작 부분에만 있는 공백 문자만 제거하며, rstrip은 문자열의 마지막 부분에 있는 공백 문자만 제거한다.|
|split|문자열을 구분자를 기준으로 부분문자열의 리스트로 분리한다.|
|lower|알파벳 문자를 소문자로 변환한다.|
|upper|알파벳 문자를 대문자로 변환한다.|
|casefold|문자를 소문자로 변환한다. 지역 문자들은 그에 상응하는 대체 문자로 교체된다.|
|ljust, rjust|문자열을 오른쪽 또는 왼쪽으로 정렬하고 주어진 길이에서 문자열의 길이를 제외한 나머지 부분은 공백 문자를 채워 넣는다.|
    
</div>
</details>

### 7.3.2 정규 표현식

- 정규 표현식은 텍스트에서 문자열 패턴을 찾는 유연한 방법을 제공
- 파이썬에서는 re 모듈이 내장되어 있어 문자열에 대한 정규 표현식을 처리함.
    - re 모듈 함수는 패턴 매칭, 치환, 분리 세 가지로 나눌 수 있음

In [68]:
import re

text = "foo bar\t baz \tqux"

re.split("\s+", text)

['foo', 'bar', 'baz', 'qux']

> re.split("\s+", text)를 사용하면 먼저 정규 표현식이 `컴파일`되고 그 다음에 split 메서드가 실행됨.  
> re.compile로 직접 정규 표현식을 컴파일하고 그렇게 얻은 정규 표현식 객체를 재사용하는 것도 가능함.  
    - 같은 정규 표현식을 다른 문자열에도 적용해야 한다면 re.compile을 이용해 CPU 사용량을 아낄 수 있다.

In [70]:
regex = re.compile("\s+")
regex.split(text)

['foo', 'bar', 'baz', 'qux']

> 정규 표현식에 매칭되는 모든 패턴의 목록을 얻고 싶다면 findall 메서드를 사용한다.

In [71]:
regex.findall(text)

[' ', '\t ', ' \t']

> findall : 문자열에서 일치하는 모든 부분문자열을 찾아줌.    
> search : 패턴과 일치하는 첫 번째 존재를 반환.  
> match : 문자열의 시작 부분에서 일치하는 것만 찾아줌.  

In [73]:
text = '''Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
'''

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

# re.IGNORECASE는 정규 표현식이 대소문자를 가리지 않도록 한다.
regex = re.compile(pattern, flags = re.IGNORECASE)

> findall : 문자열에서 일치하는 모든 주소를 찾아줌

In [74]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

> search는 텍스트에서 첫 번째 이메일 주소만 찾아줌.  
> 아래 정규 표현식에 대한 match 객체는 그 정규 표현 패턴이 문자열 내에서 위치하는 시작점과 끝점만 알려줌.

In [79]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [80]:
text[m.start():m.end()]

'dave@google.com'

> regex.match는 None을 반환함.  
> 왜나하면 그 정규 표현 패턴이 문자열의 시작점에서부터 일치하는지 검사하기 때문.

In [82]:
print(regex.match(text))

None


> sub 메서드는 찾은 패턴을 주어진 문자열로 치환하여 새로운 문자열을 반환함.

In [84]:
print(regex.sub("REDACTED", text))

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



> 이메일 주소를 찾아서 동시에 각 이메일 주소를 사용자 이름, 도메인 이름, 도메인 접미사 세 가지 컴포넌트로 나눠야 한다면 각 패턴을 괄호로 묶어준다.

In [86]:
pattern = r'([A-Z0-9._&+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

regex = re.compile(pattern, flags = re.IGNORECASE)

> 이렇게 만든 match 객체를 이용하면 groups 메서드로 각 패턴 컴포넌트의 `튜플`을 얻을 수 있다.

In [88]:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

> 패턴에 그룹이 존재한다면 findall 메서드는 튜플의 목록을 반환함.

In [89]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

> sub 역시 마찬가지로 \1, \2 같은 특수한 기호를 사용해서 각 패턴 그룹에 접근할 수 있다.

In [94]:
print(regex.sub(r'Username : \1, Domain : \2, Suffix : \3',text))

Dave Username : dave, Domain : google, Suffix : com
Steve Username : steve, Domain : gmail, Suffix : com
Rob Username : rob, Domain : gmail, Suffix : com
Ryan Username : ryan, Domain : yahoo, Suffix : com



#### 정규 표현식 메서드

<details>
<summary>정규 표현식 메서드</summary>
<div markdown="1">

|메서드|설명|
|:--|:--|
|findall|문자열에서 겹치지 않는 모든 발견된 패턴을 `리스트`로 반환함.|
|finditer|findall과 같지만 발견된 패턴을 이터레이터를 통해 하나씩 반환함.|
|match|문자열의 시작점부터 패턴을 찾고 선택적으로 패턴 컴포넌트를 그룹으로 나눈다. 일치하는 패턴이 있다면 match 객체를 반환하고 그렇지 않으면 None을 반환한다.|
|search|문자열에서 패턴과 일치하는 내용을 검색하고 match 객체를 반환한다. match 메서드와는 다르게 시작부터 일치하는 내용만 찾지 않고 문자열 어디든 일치하는 내용이 있다면 반환한다.|
|split|문자열에서 패턴과 일치하는 부분을 분리한다.|
|sub, subn|문자열에서 일치하는 모든 패턴(sub) 혹은 처음 n개의 패턴(subn)을 대체 표현으로 치환한다. 대체 표현 문자열은 \1,\2,... 와 같은 기호를 사용해서 매치 그룹의 요소를 참조한다.|
    
</div>
</details>

### 7.3.3 pandas의 벡터화된 문자열 함수

In [98]:
data= {'Dave' : 'dave@google.com',
       'Steve' : 'steve@gmail.com',
       'Rob' : 'rob@gmail.com',
       'Wes' : np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

> 문자열과 정규 표현식 메서드는 data.map을 사용해서 각 값에 적용할 수 있지만 NA 값을 만나면 실패함.  
> 이런 문제에 대처하기 위해 Series에는 NA 값을 건너뛰도록 하는 간결한 문자열 처리 메서드가 있음.  
> 이는 Series의 str속성을 이용함

In [99]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

> 정규 표현식을 IGNORANCE 같은 re 옵션과 함께 사용하는 것도 가능함.

In [105]:
pattern 

'([A-Z0-9._&+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [106]:
data.str.findall(pattern, flags = re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

> 백터화된 요소를 꺼내올 경우 str.get을 이용하거나 str 속성의 색인을 이용함.

In [113]:
matches = data.str.match(pattern, flags = re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [131]:
data.str.get(0)

Dave       d
Steve      s
Rob        r
Wes      NaN
dtype: object

In [133]:
data.str[0]

Dave       d
Steve      s
Rob        r
Wes      NaN
dtype: object

In [134]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

#### 벡터화된 문자열 메서드


<details>
<summary>벡터화된 문자열 메서드</summary>
<div markdown="1">

|메서드|설명|
|:--|:--
|cat|선택적인 구분자와 함께 요소별로 문자열을 이어붙인다.|
|contains|문자열이 패턴이나 정규 표현식을 포함하는지 나타내는 불리언 배열을 반환한다.|
|count|일치하는 패턴 수를 반환한다.|
|extract|문자열이 담긴 Series에서 하나 이상의 문자열을 추출하기 위해 정규 표현식을 이용한다. 결과는 각 그룹이 하나의 컬럼이 되는 DataFrame이다.|
|endswith|각 요소에 대해 x.endswith(pattern)과 동일한 동작을 한다.|
|startswith|각 요소에 대해 x.startswith(pattern)과 동일한 동작을 한다.|
|findall|각 문자열에 대해 일치하는 패턴/정규 표현식의 전체 목록을 구한다.|
|get|i 번째 요소를 반환한다.|
|isalnum|내장 함수 str.isalnum과 동일|
|isalpha|내장 함수 str.isalpha와 동일|
|isdecimal|내장 함수 str.isdecimal과 동일|
|isdigit|내장 함수 str.isdigit와 동일|
|islower|내장 함수 str.islower와 동일|
|isnumeric|내장 함수 str.isnumeric과 동일|
|isupper|내장 함수 str.isupper와 동일|
|join|Series의 각 요소를 주어진 구분자로 연결한다.|
|len|각 문자열의 길이를 구한다. (예, data.str.len())|
|lower, upper|대소문자로 변환한다. 각 요소에 대한 x.lower(), x.upper()와 같다.|
|match|주어진 정규 표현식으로 각 요소에 대한 re.match를 수행하여 일치하는 그룹을 리스트로 반환한다.|
|center|pad(side='both')와 동일|
|repeat|값을 복사한다. 예를 들어 s.str.repeat(3)은 각 문자열에 대한 x*3과 동일하다.|
|replace|패턴/정규 표현식과 일치하는 내용을 다른 문자열로 치환한다.|
|slice|Series 안에 있는 각 문자열을 자른다.|
|split|정규 표현식 혹은 구분자로 문자열을 나눈다.|
|strip|개행 문자를 포함하여 왼쪽과 오른쪽의 공백 문자를 제거한다.|
|rstrip|오른쪽의 공백 문자를 제거한다.|
|lstrip|왼쪽의 공백 문자를 제거한다.|
    
</div>
</details>